In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
from glob import glob
from tqdm import tqdm
import os

In [24]:
# df = pd.read_csv('tardis_data/combined/NEXOUSDT_2024_Q1.csv.gz', compression='gzip')

# # Drop unused columns
# df = df.drop(columns=['exchange', 'local_timestamp', 'id'])

# # Convert timestamp from microseconds
# df['timestamp'] = pd.to_datetime(df['timestamp'], unit='us')

# # Reorder columns
# df = df[['timestamp', 'symbol', 'side', 'price', 'amount']]

# # Set timestamp as index
# df = df.set_index('timestamp').sort_index()

# # Resample to 1-minute intervals
# df_1min = df.resample('5min').agg({
#     'price': 'last',
#     'amount': 'sum',
#     'side': 'last',
#     'symbol': 'last'
# }).dropna().reset_index()

In [25]:
# plt.plot(df_1min['price'].head(100), color='blue', label='Price')
# plt.xlabel('Time')
# plt.ylabel('Price')
# plt.title('Price over Time')
# plt.legend()
# plt.show()

In [26]:
# df_liquid = pd.read_csv('tardis_data/combined/NEXOUSDT_2024_Q1.csv.gz', compression='gzip')

# # Drop unused columns
# df_liquid = df_liquid.drop(columns=['exchange', 'local_timestamp', 'id'])

# # Convert timestamp from microseconds
# df_liquid['timestamp'] = pd.to_datetime(df_liquid['timestamp'], unit='us')

# # Reorder columns
# df_liquid = df_liquid[['timestamp', 'symbol', 'side', 'price', 'amount']]

# # Set timestamp as index
# df_liquid = df_liquid.set_index('timestamp').sort_index()

# # Resample to 1-minute intervals
# df_liquid_1min = df_liquid.resample('5min').agg({
#     'price': 'last',
#     'amount': 'sum',
#     'side': 'last',
#     'symbol': 'last'
# }).dropna().reset_index()

In [27]:
# plt.plot(df_liquid_1min['price'].head(100), color='blue', label='Price')

In [28]:
# #Count the number of times the price does not change over a 1-minute, 2-minute, and 5-minute interval

# # Count 1-minute price holds
# df_1m = df.set_index('timestamp').resample('1min').last().dropna().reset_index()
# no_change_1m = (df['price'].diff() == 0).sum()

# # Count 3-minute price holds
# df_3m = df.set_index('timestamp').resample('3min').last().dropna().reset_index()
# no_change_3m = (df_3m['price'].diff() == 0).sum()

# # Count 5-minute price holds
# df_5m = df.set_index('timestamp').resample('5min').last().dropna().reset_index()
# no_change_5m = (df_5m['price'].diff() == 0).sum()

# #Count 30-minute price holds
# df_30m = df.set_index('timestamp').resample('30min').last().dropna().reset_index()
# no_change_30m = (df_30m['price'].diff() == 0).sum()

# print(f"Price unchanged:")
# print(f"  Over 1-minute intervals: {no_change_1m}")
# print(f"  Over 3-minute intervals: {no_change_3m}")
# print(f"  Over 5-minute intervals: {no_change_5m}")
# print(f"  Over 30-minute intervals: {no_change_30m}")


In [29]:
# def combine_tardis_trades(file_paths, output_path=None):
#     """
#     Combines multiple Tardis trades files (compressed .csv.gz) into a single DataFrame
#     with 1-minute resampled last prices for each symbol.

#     Parameters:
#     - file_paths (list): List of full paths to the .csv.gz files.
#     - output_path (str, optional): Path to save the final combined CSV file.

#     Returns:
#     - pd.DataFrame: Combined DataFrame with timestamp and resampled price columns per symbol.
#     """

#     resampled_dfs = []

#     print("📊 Processing trade files:")
#     for file_path in tqdm(file_paths, desc="Files processed"):
#         # Read file
#         df = pd.read_csv(file_path, compression='gzip')

#         # Drop unused columns
#         df = df.drop(columns=['exchange', 'local_timestamp', 'id'])

#         # Convert timestamp from microseconds
#         df['timestamp'] = pd.to_datetime(df['timestamp'], unit='us')

#         # Reorder columns
#         df = df[['timestamp', 'symbol', 'side', 'price', 'amount']]

#         # Set timestamp as index
#         df = df.set_index('timestamp').sort_index()

#         # Resample to 1-minute intervals
#         df_1min = df.resample('5min').agg({
#             'price': 'last',
#             'amount': 'sum',
#             'side': 'last',
#             'symbol': 'last'
#         }).dropna().reset_index()

#         # Rename price column to last_price
#         df_1min = df_1min.rename(columns={'price': 'last_price'})

#         # Rename for final merge: SYMBOL/USDT_2024_1m
#         symbol = df_1min['symbol'].iloc[0]
#         df_1min = df_1min.rename(columns={'last_price': f'{symbol}_2024_5m'})

#         # Reduce to timestamp + price column
#         df_1min = df_1min[['timestamp', f'{symbol}_2024_5m']]

#         # Store for merging later
#         resampled_dfs.append(df_1min)

#     print("🔗 Merging DataFrames...")
#     df_combined = reduce(lambda left, right: pd.merge(left, right, on='timestamp', how='inner'), resampled_dfs)

#     if output_path:
#         df_combined.to_csv(output_path, index=False, compression='gzip')
#         print(f"✅ Final combined file saved to {output_path}")

#     return df_combined

def combine_tardis_trades(file_paths, output_path=None):
    """
    Combines large Tardis trades files into a single DataFrame with 5-minute
    resampled last prices per symbol. Optimized for low memory usage.
    """

    combined_df = None  # Start with nothing

    print("📊 Processing and merging trade files one-by-one:")
    for file_path in tqdm(file_paths, desc="Files processed"):
        # Read and process each file
        df = pd.read_csv(file_path, compression='gzip')

        df = df.drop(columns=['exchange', 'local_timestamp', 'id'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='us')
        df = df[['timestamp', 'symbol', 'side', 'price', 'amount']]
        df = df.set_index('timestamp').sort_index()

        df_1min = df.resample('5min').agg({
            'price': 'last',
            'amount': 'sum',
            'side': 'last',
            'symbol': 'last'
        }).dropna().reset_index()

        symbol = df_1min['symbol'].iloc[0]
        price_col = f'{symbol}_2024_5m'
        df_1min = df_1min.rename(columns={'price': price_col})
        df_1min = df_1min[['timestamp', price_col]]
        
        #Merge is now done incrementally
        # Merge into combined_df
        if combined_df is None:
            combined_df = df_1min
        else:
            combined_df = pd.merge(combined_df, df_1min, on='timestamp', how='inner')

    # Check if output path exits and if not, create it
    if output_path:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        combined_df.to_csv(output_path, index=False, compression='gzip')
        print(f"✅ Final combined file saved to {output_path}")


    

    return combined_df

In [30]:
# #Create a function to calculate the daily average trading volume for each ticker

# def calculate_average_daily_volume(file_paths, output_path=None):

#     """
#     Calculates the daily average trading volume for each ticker from multiple Tardis trades files.

#     Parameters:
#     - file_paths (list): List of full paths to the .csv.gz files.
#     - output_path (str, optional): Path to save the final combined CSV file.

#     Returns:
#     - pd.DataFrame: DataFrame with daily average volume for each ticker.
#     """

#     average_daily_volumes = []

#     print("📊 Processing trade files for daily volume:")
#     for file_path in tqdm(file_paths, desc="Files processed"):

#         #Create a DataFrame with the ticker name and average daily volume
#         ticker = file_path.split('/')[-1].split('_')[0]

#         print("PROCESSING TICKER :", ticker)

#         # Read file
#         df = pd.read_csv(file_path, compression='gzip')

#         # Drop unused columns
#         df = df.drop(columns=['exchange', 'local_timestamp', 'id'])

#         # Convert timestamp from microseconds
#         df['timestamp'] = pd.to_datetime(df['timestamp'], unit='us')

#         # Calculate USD volume
#         df['usd_volume'] = df['price'] * df['amount']

#         # Resample to 1-day intervals and sum volumes
#         daily_volumes = df.set_index('timestamp')['usd_volume'].resample('1D').sum().reset_index()

#         #Calculate the average daily volume
#         average_daily_volume = daily_volumes['usd_volume'].mean()

#         # print(f"Average daily volume for {ticker} below:")
#         # print(average_daily_volume)

#         average_daily_volume_df = pd.DataFrame({
#             'ticker': ticker,
#             'average_daily_volume': average_daily_volume
#         }, index=[0])

#         # Store for merging later
#         average_daily_volumes.append(average_daily_volume_df)


#     print("🔗 Merging DataFrames...")
#     #Combine all average daily volumes for each ticker into a single DataFrame

#     #Concatenate all DataFrames
#     df_combined_volumes = pd.concat(average_daily_volumes, ignore_index=True)

#     # Check if output_path exists and if not, create it
#     output_dir = os.path.dirname(output_path)
#     if output_dir and not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     df_combined_volumes.to_csv(output_path, index=False, compression='gzip')
#     print(f"✅ Final combined volume file saved to {output_path}")


#     return df_combined_volumes


In [31]:
# # Get all your combined files
# file_paths = glob("tardis_data/combined/*_2024_Q1.csv.gz") #Glob matches all files in the directory that end with _2024_Q1.csv.gz

# df_combined_volumes = calculate_average_daily_volume(file_paths, output_path='tardis_data/volumes/average_daily_volumes_2024_Q1.csv.gz')


In [32]:
# volumes = pd.read_csv('tardis_data/volumes/average_daily_volumes_2024_Q1.csv.gz', compression='gzip')

# #Order volumes by average daily volume
# volumes = volumes.sort_values(by='average_daily_volume', ascending=False)

# #Print entire DataFrame with no truncation
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# volumes

In [33]:
# Get all your combined files
file_paths = glob("tardis_data/combined/*_Jan_To_December.csv.gz") #Glob matches all files in the directory that end with _2024_Q1.csv.gz

# Combine and save
df_final = combine_tardis_trades(file_paths, output_path="tardis_data/combined_JAN_To_DEC/combined_2024_new_5m.csv.gz")


📊 Processing and merging trade files one-by-one:


Files processed: 100%|██████████| 4/4 [01:09<00:00, 17.37s/it]


✅ Final combined file saved to tardis_data/combined_JAN_To_DEC/combined_2024_new_5m.csv.gz


In [34]:
# df_final.to_csv('tardis_data/combined_October_To_March/combined_2024_2025_5m.csv', index=False)

In [45]:
new_df = pd.read_csv("tardis_data/combined_JAN_To_DEC/combined_2024_new_5m.csv.gz", compression='gzip')

old_df = pd.read_csv("tardis_data/final_training_set/final_training_set_5min_2024.csv")

#Merge the two DataFrames on timestamp using an inner join
merged_df = pd.merge(new_df, old_df, on='timestamp', how='inner')
#Save the merged DataFrame to a new CSV file
merged_df.to_csv("tardis_data/final_training_set/final_training_set_top_10.csv", index=False)


In [44]:
old_df

,timestamp,SANDUSDT_2024_5m,MANAUSDT_2024_5m,AXSUSDT_2024_5m
0,2024-01-01 00:00:00,0.6006,0.5239,8.880
1,2024-01-01 00:05:00,0.5971,0.5228,8.870
2,2024-01-01 00:10:00,0.5979,0.5228,8.870
3,2024-01-01 00:15:00,0.5941,0.5213,8.850
4,2024-01-01 00:20:00,0.5947,0.5218,8.850
...,...,...,...,...
103612,2024-12-31 23:35:00,0.5445,0.4651,6.195
103613,2024-12-31 23:40:00,0.5451,0.4661,6.199
103614,2024-12-31 23:45:00,0.5466,0.4669,6.214
103615,2024-12-31 23:50:00,0.5461,0.4669,6.219


In [35]:
# #Final Training set
# #---
# data_jan_to_june = pd.read_csv('tardis_data/final_in_sample_dataset/final_in_sample_dataset_5min_2024.csv', parse_dates=True)
# data_jan_to_june = data_jan_to_june[['timestamp','SANDUSDT_2024_5m','MANAUSDT_2024_5m', 'AXSUSDT_2024_5m']]

# data_july_to_september = pd.read_csv('tardis_data/final_out_of_sample_dataset_5min/final_out_of_sample_dataset_5min.csv', parse_dates=True)

# data_october_to_march = pd.read_csv('tardis_data/combined_October_To_March/combined_2024_2025_5m.csv.gz', parse_dates=True)

# #Arrange columns in the same order as the training set
# data_october_to_march = data_october_to_march[['timestamp', 'SANDUSDT_2024_5m', 'MANAUSDT_2024_5m', 'AXSUSDT_2024_5m']]

# data_october_to_december = data_october_to_march[data_october_to_march['timestamp'] < '2025-01-01']

# final_training_set = pd.concat([data_jan_to_june, data_july_to_september, data_october_to_december], ignore_index=True)

# final_training_set.to_csv('tardis_data/final_training_set/final_training_set_5min_2024.csv', index=False)



In [36]:
# final_training_set_read = pd.read_csv('tardis_data/final_training_set/final_training_set_5min_2024.csv', parse_dates=True)
# final_training_set_read

In [37]:
# #Final Testing set
# #---

# data_january_to_march_2025 = data_october_to_march[data_october_to_march['timestamp'] >= '2025-01-01']
# final_test_set = data_january_to_march_2025
# final_test_set.to_csv('tardis_data/final_testing_set/final_test_set_5min_2025.csv', index=False)

In [38]:
# final_test_set_read = pd.read_csv('tardis_data/final_testing_set/final_test_set_5min_2025.csv', parse_dates=True)
# final_test_set_read